# Feature Selection using ReliefF with scikit-rebate

Adapted from: https://github.com/PadraigC/FeatSelTutorial.git

In [ ]:
pip install skrebate

In [ ]:
import pandas as pd
import numpy as np
from skrebate import ReliefF
from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt 

Load the data, scale it and divide into train and test sets.  
The filters are *trained* using the training data and then a classifier is trained on the feature subset and tested on the test set. 

In [ ]:
import pandas as pd

df = pd.read_csv("data/train_data.csv", index_col=0)

df = df.dropna()

df['a'] = pd.to_datetime(df['a']).astype('int64')
df['b'] = pd.to_datetime(df['b']).astype('int64')
df['c'] = pd.to_datetime(df['c']).astype('int64')

X = df.drop(["to_be_predicted"], axis=1)
X = df.select_dtypes(include=[int,float])
y = df.pop("to_be_predicted").values


In [ ]:
X_raw = X.values

X_tr_raw, X_ts_raw, y_train, y_test = train_test_split(X_raw, y, 
                                                       random_state=42, test_size=1/2)
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_tr_raw)
X_test = scaler.transform(X_ts_raw)

feature_names = X.columns
X_train.shape, X_test.shape

### ReliefF
- `ReliefF` will produce scores for all features.
- `n_features_to_select` controls the transform behaviour, if a dataset is transformed this number of features will be retained. 

In [ ]:
reliefFS = ReliefF(n_features_to_select=11, n_neighbors=100, n_jobs = -1)

In [ ]:
reliefFS.fit(X_train,y_train)

In [ ]:
relief_scores = reliefFS.feature_importances_

In [ ]:
reliefFS.transform(X_train).shape

Also calcuate I-Gain scores: to be used for comparision.

In [ ]:
i_scores = mutual_info_classif(X_train,y_train)
i_scores
# The i-gain scores for the features

In [ ]:
from scipy import stats
stats.spearmanr(relief_scores, i_scores)

Store the ReliefF and I-Gain scores in a dataframe.  

In [ ]:
dtf=pd.DataFrame({'Mutual Info.':i_scores,'ReliefF':relief_scores,'Feature':feature_names})
dtf.set_index('Feature', inplace = True)
dtf.sort_values('Mutual Info.', inplace = True, ascending = False)
dtf

### Plotting the ReliefF and I-Gain scores

In [ ]:
fig, ax = plt.subplots()
rr = range(0,len(feature_names))
ax2 = ax.twinx()
ax.plot(dtf.index, dtf["Mutual Info."], label='I-Gain')
ax2.plot(dtf.index, dtf["ReliefF"], color='red', label='Relief')
ax.set_xticks(rr)

ax.set_xticklabels(list(dtf.index), rotation = 90)
ax.set_xlabel('Features', fontsize=12, fontweight='bold')
ax.set_ylabel('I-Gain')
ax2.set_ylabel('ReliefF')
fig.legend(loc="upper right", bbox_to_anchor=(1,1), bbox_transform=ax.transAxes)